In [1]:
from rank_bm25 import BM25Okapi, BM25Plus
from elasticsearch import Elasticsearch

from transformers import AutoTokenizer
import os
import json
import pickle
import numpy as np
import pandas as pd
import re
from tqdm.auto import tqdm

from datasets import load_from_disk

In [2]:
def preprocess_retrieval(corpus):
    corpus = corpus.replace("\\n", "")
    corpus = re.sub(f"[^- ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Zぁ-ゔァ-ヴー々〆〤一-龥]", " ", corpus)
    corpus = ' '.join(corpus.split())
    return corpus

In [3]:
train_dataset = load_from_disk("../data/train_dataset")
train_context = []
train_query = []
for data in tqdm(train_dataset['train']):
    train_context.append(data['context'])
    train_query.append(data['question'])

In [46]:
initial_contexts = list(set(train_context))
contexts = [preprocess_retrieval(corpus) for corpus in initial_contexts]
len(contexts), len(initial_contexts)

(3340, 3340)

In [5]:
def make_elasticsearch(contexts, index_name):
    os.system("service elasticsearch start")
    INDEX_NAME = index_name

    INDEX_SETTINGS = {"settings" : {"index":{"analysis":{"analyzer":{"korean":{"type":"custom",
                                            "tokenizer":"nori_tokenizer","filter": [ "shingle" ],}}}}},
      "mappings": {"properties" : {"context" : {"type" : "text","analyzer": "korean","search_analyzer": "korean"},}}}
    
    DOCS = {}
    for i in tqdm(range(len(contexts))):
        DOCS[i] = {'context':contexts[i]}
        
    try:
        es.transport.close()
    except:
        pass
    es = Elasticsearch()
    
    if es.indices.exists(INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
    es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)
    
    for doc_id, doc in tqdm(DOCS.items()):
        es.index(index=INDEX_NAME,  id=doc_id, body=doc)
        
    return es

In [10]:
es = make_elasticsearch(contexts, "wiki_index")

 * Starting Elasticsearch Server
 * Already running.
   ...done.


<ipython-input-5-708f6d5b1640>:19: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-5-708f6d5b1640>:21: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NA

<ipython-input-5-708f6d5b1640>:24: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


In [42]:
def retrieval_es(model, query, index_name, size=1):
    try:
        res = model.search(index=index_name, q=query, size=size)
    except:
        mod_q = query.replace("%", " ").replace("-", " ")
        res = model.search(index=index_name, q=mod_q, size=size)
    
    doc_score = [float(res['hits']['hits'][idx]['_score']) for idx in range(size)]
    doc_indices = [int(res['hits']['hits'][idx]['_id']) for idx in range(size)]
    return doc_score, doc_indices

In [62]:
def negative_samples(model, contexts, passage, query, index_name, topk=1):
    p_with_neg = []
    double_size = topk*2
    scores, indices = retrieval_es(model, query, index_name, size=double_size)
    # p_with_neg.append(passage)
    for idx in indices:
        if contexts[idx] != passage and contexts[idx] not in p_with_neg:
            p_with_neg.append(contexts[idx])
        if len(p_with_neg) == topk:
            break
    return p_with_neg    

In [63]:
p_with_negs = []
for i in tqdm(range(len(train_query))):
    sample = negative_samples(es, initial_contexts, train_context[i], preprocess_retrieval(train_query[i]), "wiki_index", topk=3)
    p_with_negs.append(sample)
len(p_with_negs), p_with_negs[0]

(3952,
 ["미국의 외교정책의 수립과 이행에 대한 주된 책임을 대통령이 지게 되어 있으나, 의회도 이에 대해 강한 통제력을 발휘할 수 있다. 우선 의회는 전쟁을 선포할 수 있는 권한이 있다. 하원은 대통령의 외교정책 수행에 필요한 예산에 대해 강한 통제력을 가지고 있으며, 상원도 재원에 대한 통제가 가능하다. 특히 상원의 경우에는 고급 외무 관리의 임명에 대한 통제력을 가지고 있을 뿐만 아니라, 미국이 체결하는 모든 조약에 대한 비준권을 가지고 있다. 의회의 이와 같은 제재 권한은 행정부와 의회의 갈등을 초래하는 요소로 작용한다. 에드워드 코르윈(Edward Samuel Corwin) 교수는 미국의 정치체계가 외교정책에 있어서 행정부와 의회 간의 투쟁을 초래한다고 지적한 바 있다. 그 좋은 예로서는 상원이 국제연맹 헌장의 비준을 거부한 사실을 들 수 있다. 의회와 행정부간에는 긴밀한 협조가 이루어지고 있다. 유엔헌장 비준 당시의 의회와 행정부 간의 긴밀한 협조는 너무나 유명하다. 그러나 이러한 협동외교는 한국에서 흔히 사용되고 있는 '초당외교'라는 말과는 엄격히 구분되어야 한다. 베트남 전쟁으로 말미암아 파생된 행정부와 의회 간의 갈등은 심각한 형편이었다. 그러나 의회가 대통령에 가하는 압력은 행정부의 독주를 견제하는 동시에 건전한 방향의 미국 대외정책을 모색하고 있다. 의회는 국민의 대표기구로서 국민의 의사와 직결돼 있다. 따라서 대외정책의 수립이나 이행 과정에도 항상 의회를 '문제아'로 취급하거나, 또는 그러한 범주 내에 의회의 기능을 이해해서는 안 된다.",
  '비건은 1984년 미시간 대학교에서 러시아어와 정치학 학사 학위를 받았다. 1992년부터 1994년까지는 국제 공화주의 연구소의 러시아 모스크바 지국장으로 있었다. 조지 W. 부시 행정부 시절 미국 국가안전보장회의 사무국장으로 재직하며 국가안보보좌관 콘돌리자 라이스를 보좌했고, 2008년 미국 대통령 선거에서 존 매케인의 부통령 후보 세라 페일린의 외교정책보좌관을 맡았다.\\n\\n2018

In [64]:
for i in range(len(p_with_negs)):
    assert len(p_with_negs[0]) == 3, "Wrong Topk"

In [65]:
train_dataset['train']

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [66]:
index_level = list(train_dataset['train']['__index_level_0__'])
answers = list(train_dataset['train']['answers'])
document_id = list(train_dataset['train']['document_id'])
ids = list(train_dataset['train']['id'])
title = list(train_dataset['train']['title'])

In [67]:
df = pd.DataFrame({'__index_level_0__':index_level,
                  'answers':answers,
                  'context':train_context,
                  'document_id':document_id,
                  'id':ids,
                  'question':train_query,
                  'title':title,
                  'nagative_samples':p_with_negs})

In [68]:
df.to_csv("../negative_samples.csv", index=False)